# Plan-Excute Agent  学习
1. 定义 tools
2. 定义 model, memory
3. 定义 nodes
4. 定义 graph，用 edge 把 node 连成图
5. 打印执行计划并确认
6. 执行计划

In [1]:
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END, MessagesState
from typing import List
import json

## 1、定义 Tools

In [2]:
@tool
def search_companies_by_industry(industry: str) -> str:
    """输入行业名称，返回一个公司列表ID。必须先用此工具获取 ID，才能用 search_people_by_company_list 查询人员。"""
    return "company_list_123"

@tool
def get_company_list_id_from_names(companies: list) -> str:
    """输入公司名称列表，返回一个公司列表ID，用于后续批量查询人员。"""
    return "company_list_from_names_456"
    
@tool
def expand_job_title(base_title: str) -> list:
    """
    使用大模型，根据一个职位名称生成相关职位名称列表。
    例如: "软件工程师" → ["软件工程师", "后端开发", "全栈工程师", "研发工程师"]
    """
    llm = init_chat_model("gpt-4o-mini", temperature=0)
    prompt = f"""
    你是一个人力资源专家。给定职位名称 "{base_title}"，
    请生成一个包含 3~5 个相关职位名称的 JSON 数组。
    只输出 JSON 数组，不要多余文字，不要用 markdown。
    """
    resp = llm.invoke(prompt)
    try:
        print('expand_job_title:', resp.content)
        import json
        job_list = json.loads(resp.content)
        if isinstance(job_list, list):
            return job_list
    except Exception:
        raise
    return [base_title]  # fallback

@tool
def search_people_by_company_list(company_list_id: str, job_titles: list) -> list:
    """
    输入公司列表ID和多个职位关键字，批量查找这些公司的人。
    job_titles 示例: ["软件工程师", "后端开发", "研发工程师"]
    """
    results = []
    for title in job_titles:
        results.append({
            "name": f"Test-{title}",
            "title": title,
            "company": "DemoCorp"
        })
    return results    

@tool
def get_phone_number_by_name(name: str) -> str:
    """输入一个人的姓名，返回手机号。"""
    phone_book = {
        "Test-软件工程师": "13800000001",
        "Test-后端开发": "13800000002",
        "Test-全栈工程师": "13800000003",
        "Test-研发工程师": "13800000004"
    }
    return phone_book.get(name, "未知")    

## 2、定义 Model, Memory

In [3]:
# ====== 创建模型 & 工具列表 ======
tools = [
    search_companies_by_industry,
    get_company_list_id_from_names,
    expand_job_title,
    search_people_by_company_list,
    get_phone_number_by_name
]
model = init_chat_model("gpt-4o-mini", temperature=0).bind_tools(tools)
memory = MemorySaver()


## 3、定义 plan 和 execute 节点

In [4]:
# ------- plan 节点 -------
def plan_node(state: MessagesState):
    user_query = state["messages"][-1].content
    plan_prompt = f"""
    用户需求：{user_query}
    请一步步推理需要调用哪些工具，每个工具的输入是什么，以及调用顺序。
    输出 JSON 数组，每个元素包含 "tool", "input", "purpose" 三个字段。
    不要执行工具，只规划，不要输出 markdown 标记。
    """
    resp = model.invoke([("system", "你是一个任务规划助手"), ("user", plan_prompt)])
    plan_text = resp.content
    print("\n=== AI 任务计划 ===\n")
    print(plan_text)
    return {"messages": state["messages"] + [resp], "plan_text": plan_text, "user_query": user_query}


# ------- execute 节点 -------
def execute_node(state: MessagesState):
    print("\n=== 开始执行计划 ===\n")
    # 直接调用 ReAct agent 模式
    # 这里复用相同的 model & tools，上下文依然保留
    from langgraph.prebuilt import create_react_agent
    agent = create_react_agent(model, tools, checkpointer=memory)
    config = {"configurable": {"thread_id": "thread-1"}}
    
    events = agent.stream({"messages": state["messages"]}, config)
    
    plan_text = state.get("plan_text", '')
    user_query = state.get("user_query", '')
    prompt = f"""
    用户的问题是：{user_query}
    请严格按照以下计划执行，不要生成新的计划：
    {plan_text}
    """
    events = agent.stream({"messages": [("user", prompt)]}, config)
    
    for event in events:
        if "agent" in event:
            for msg in event["agent"]["messages"]:
                msg.pretty_print()
        if "tools" in event:
            for msg in event["tools"]["messages"]:
                msg.pretty_print()
    return state

## 4、定义 Grahp，将点和边连起来

In [5]:
graph = StateGraph(MessagesState)
# ====== 添加节点到图 ======
graph.add_node("plan", plan_node)
graph.add_node("execute", execute_node)

graph.add_edge("plan", "execute")
graph.add_edge("execute", END)

graph.set_entry_point("plan")
app = graph.compile(checkpointer=memory)

## 5、打印执行计划并确认

In [6]:
# ===== Stream 执行 =====
user_input = "帮我查这些公司里的软件工程师并给出手机号：阿里巴巴, 腾讯"
config = {"configurable": {"thread_id": "thread-1"}}

# 第一次只跑到 plan 节点，然后停下来
events = app.stream({"messages": [("user", user_input)]}, config, stop_at=["plan"])

state_after_plan = None
for event in events:
    if "plan" in event:
        state_after_plan = app.get_state(config)
        break


=== AI 任务计划 ===

[
    {
        "tool": "functions.get_company_list_id_from_names",
        "input": {
            "companies": ["阿里巴巴", "腾讯"]
        },
        "purpose": "获取阿里巴巴和腾讯的公司列表ID，以便后续查询人员。"
    },
    {
        "tool": "functions.expand_job_title",
        "input": {
            "base_title": "软件工程师"
        },
        "purpose": "生成与软件工程师相关的职位名称列表，以便进行更全面的人员查询。"
    },
    {
        "tool": "functions.search_people_by_company_list",
        "input": {
            "company_list_id": "从第一步获取的公司列表ID",
            "job_titles": "从第二步生成的职位名称列表"
        },
        "purpose": "根据公司列表ID和职位名称查询软件工程师及相关职位的人员信息。"
    },
    {
        "tool": "functions.get_phone_number_by_name",
        "input": {
            "name": "从第三步获取的人员姓名"
        },
        "purpose": "根据人员姓名获取他们的手机号。"
    }
]


## 6、执行计划得到结果

In [7]:
# 确认是否执行
if state_after_plan:
    # confirm = input("\n是否执行计划？(y/n)：").strip().lower()
    confirm = 'y' # for test
    if confirm == "y":
        for event in events:
            if "agent" in event:
                for msg in event["agent"]["messages"]:
                    msg.pretty_print()
            if "tools" in event:
                for msg in event["tools"]["messages"]:
                    msg.pretty_print()
    else:
        print("已取消执行。")



=== 开始执行计划 ===

================================== Ai Message ==================================
Tool Calls:
  get_company_list_id_from_names (call_jSb6oV2P3b3oASXw3wMuRIwI)
 Call ID: call_jSb6oV2P3b3oASXw3wMuRIwI
  Args:
    companies: ['阿里巴巴', '腾讯']
================================= Tool Message =================================
Name: get_company_list_id_from_names

company_list_from_names_456
================================== Ai Message ==================================
Tool Calls:
  expand_job_title (call_MqEiuD8QIHr6kM5RJoOYvYjZ)
 Call ID: call_MqEiuD8QIHr6kM5RJoOYvYjZ
  Args:
    base_title: 软件工程师
expand_job_title: [
    "前端开发工程师",
    "后端开发工程师",
    "全栈工程师",
    "移动应用开发工程师",
    "软件测试工程师"
]
================================= Tool Message =================================
Name: expand_job_title

["前端开发工程师", "后端开发工程师", "全栈工程师", "移动应用开发工程师", "软件测试工程师"]
================================== Ai Message ==================================
Tool Calls:
  search_people_by_company_list (cal